### 设计一个CNN网络，训练自定义图像数据集的分类操作。 通过优化网络结构与超参数、正则化、数据增强等各种手段，尽可能提高准确率。（要求自建模型，形式不限，不能使用预定义模型）
Dataset Context：<BR>
This is image data of Natural Scenes around the world.<BR>
Dataset Content：<BR>
This Data contains around 25k images of size 96x96 distributed under 6 categories.<BR>
{'buildings' -> 0, 'forest' -> 1, 'glacier' -> 2,
'mountain' -> 3,  'sea' -> 4, 'street' -> 5 }
<BR>
    
#### 要求：
1)利用callback将最佳模型保存到文件(注意：在"save"目录下建立以自己学号命名的子目录，然后在该子目录下保存文件)。显示loss曲线和accuracy曲线。<BR>
2)读取最佳模型进行指标评估并显示结果，展示混淆矩阵。<BR>
3)尝试展示典型图片的热力图<BR>

#### 考核办法：
1）程序功能完成度<BR>
2）score = model.evaluate(testset)<BR>
计算得到的准确率为指标，达到0.8为及格成绩起点，0.9优秀<BR>

#### 数据组织形式：seg_train为训练集，seg_test为测试集，各category子目录下存有jpg文件的图片
data/project/
    ├── seg_test
    │   └── seg_test
    │       ├── buildings
    │       ├── forest
    │       ├── glacier
    │       ├── mountain
    │       ├── sea
    │       └── street
    └── seg_train
        └── seg_train
            ├── buildings
            ├── forest
            ├── glacier
            ├── mountain
            ├── sea
            └── street

#### 数据读取方法：
  IMG_SIZE=96
  datagen = tf.keras.preprocessing.image.ImageDataGenerator(...)
  trainset = datagen.flow_from_directory(trainpath, (IMG_SIZE, IMG_SIZE), batch_size=32)
  testset = datagen.flow_from_directory(testpath, (IMG_SIZE, IMG_SIZE), batch_size=32)
  
model.fit(trainset, validation_data=testset, ...)  #不区分验证集与测试集<BR>



关于数据增强，ImageDataGenerator自带很多增强方式，十分方便，请参考相关文档
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator


In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

trainpath = 'data/project/seg_train/seg_train'
testpath = 'data/project/seg_test/seg_test'
IMG_SIZE = 96
batch_size = 64

datagen = ImageDataGenerator(rescale=1./255,rotation_range=30,shear_range=0.3,
                             height_shift_range=0.2,width_shift_range=0.2)
datagen_ori = ImageDataGenerator(rescale=1./255)

trainset = datagen.flow_from_directory(trainpath, (IMG_SIZE, IMG_SIZE), batch_size=batch_size)
testset = datagen_ori.flow_from_directory(testpath, (IMG_SIZE, IMG_SIZE), 
                                          batch_size=batch_size,shuffle=False)

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D

model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(5,5),input_shape=(96,96,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64,kernel_size=(4,4),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128,kernel_size=(3,3),activation='sigmoid',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6,activation='softmax'))

model.summary()

In [3]:
from tensorflow.keras.optimizers import Adam,SGD

# opt = Adam(learning_rate=1e-3,decay=1e-3)
opt = SGD(learning_rate=0.1,decay=1e-3,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint

savepath = 'save/3200102349/project1.h5'
checkpoint = ModelCheckpoint(filepath=savepath,monitor='val_accuracy',verbose=0,save_best_only=True)

In [ ]:
history = model.fit(trainset,validation_data=testset,epochs=40,
                              steps_per_epoch=len(trainset),
                              verbose=1,shuffle=True,
                              callbacks=[checkpoint])

import matplotlib.pyplot as plt
def show_train_history(history, train, validation):
    plt.plot(history.history[train])
    plt.plot(history.history[validation])
    plt.title('train history')
    plt.ylabel(train)
    plt.xlabel('epoch')
    plt.legend(['train','validation'], loc='upper left')
    plt.grid()
    plt.show()

show_train_history(history,'accuracy','val_accuracy')
show_train_history(history,'loss','val_loss')

In [ ]:
from keras.models import load_model

bestmodel = load_model(filepath=savepath)
score = bestmodel.evaluate(testset)
print('score: %.3f' % score[1])

In [ ]:
import numpy as np

prediction = np.argmax(bestmodel.predict(testset),axis=-1)

In [ ]:
import pandas as pd

test_labels = testset.classes
pd.crosstab(test_labels,prediction,rownames=['label'],colnames=['predict'])

In [ ]:
from keras.preprocessing import image
import matplotlib.pyplot as plt

img_path = 'data/project/seg_train/seg_train/buildings/0.jpg'
img_ori = image.load_img(img_path,target_size=(96,96))
img_ori = image.img_to_array(img_ori) / 255.
plt.imshow(img_ori)
plt.show()

img = np.expand_dims(img_ori,axis=0)

In [ ]:
last_layer_weights = bestmodel.layers[-1].get_weights()[0]

from keras.models import Model
conv_model = Model(inputs=bestmodel.input,
                   outputs=(bestmodel.layers[5].output, bestmodel.layers[-1].output))

last_conv_out,img_pred = conv_model.predict(img)
last_conv_out = np.squeeze(last_conv_out)

pred_idx = np.argmax(img_pred[0])
print('prediction: %d' % pred_idx)

In [ ]:
last_layer_weights_for_pred = last_layer_weights[:,pred_idx]
import cv2
heat_map = np.dot(last_conv_out,last_layer_weights_for_pred)
heat_map = cv2.resize(heat_map,(96,96))
plt.imshow(heat_map)

In [ ]:
fig,ax = plt.subplots()
ax.imshow(img_ori)
ax.imshow(heat_map,cmap='jet',alpha=0.5)
plt.show()